# Environment Set Up

In [127]:
from collections import defaultdict
import numpy as np
import random

In [128]:
maze = np.zeros((4,4)) # represents a gridworld in which each state, apart from the terminal state, has a value of 0. 
terminal_state = [(0,0), (3,3)]

## Concept Check

What is value? The sum of all future rewards that can be recieved from a given state s. These values are "long lasting. Tells us the long term effects of being in a certain state



## Iterative Policy Evaluation for Gridworld 

What is iterative policy evaluation? Finding the true value function for each state with full knowledge of the environment

In [85]:
# calculate the value of each state given the policy
def policy_eval(maze_values): 
  
    
    new_values = maze_values.copy()
    
    for row in range(len(maze_values[0])): 
        for col in range(len(maze_values[1])):
            reward = -1 # -1 for up, down, left, right
            successor_val = 0
            initial_state_val = maze_values[row][col]
            
            if ((row, col) in terminal_state): 
                new_values[row][col] = 0
            else:
                # check left
                if (col - 1 < 0): 
                    successor_val += reward + maze_values[row][col]
                else: 
                    successor_val += (reward + maze_values[row][col - 1])
                # check right
                if (col + 1 >= len(maze_values[1])): 
                    successor_val += reward + maze_values[row][col]
                else: 
                    successor_val += (reward + maze_values[row][col + 1])
                # check up
                if (row - 1 < 0): 
                    successor_val += reward + maze_values[row][col]
                else: 
                    successor_val += (reward + maze_values[row - 1][col])
                # check down
                if (row + 1 >= len(maze_values[0])): 
                    successor_val += reward + maze_values[row][col]
                else: 
                    successor_val += (reward + maze_values[row+1][col])
                
                new_values[row][col] = 0.25 * successor_val
                
            
        
    return np.array(new_values) 
                
            
        
    

                

In [86]:
maze_values = np.zeros((4,4))

print(np.array(maze_values))

current_values = maze_values

for i in range(1000): 
    
    current_values = policy_eval(current_values)
    
    
print(current_values)



[[0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]]
[[  0. -14. -20. -22.]
 [-14. -18. -20. -20.]
 [-20. -20. -18. -14.]
 [-22. -20. -14.   0.]]


# Step Function

In [126]:
def step(pos, Q):
    
    eps = 0.5
    action = e_greedy(eps, pos, Q)
    successor_val = 0
    successor_state = pos
    reward = -1
    
    if action == 0: # check up
        if not (pos[0] - 1 < 0): 
            successor_state = (pos[0] - 1, pos[1])
    if action == 1: # check down
        if not (pos[0] + 1 > 3): 
            successor_state = (pos[0] + 1, pos[1])
    if action == 2: # check left
        if not (pos[1] - 1 < 0): 
            successor_state = (pos[0], pos[1] - 1)
    if action == 3: # check right
        if not (pos[1] + 1 > 3): 
            successor_state = (pos[0], pos[1] + 1)
    
    #if successor_state in terminal_state: 
        #reward = 0      
        
    if pos in terminal_state:
        reward = 0
                               
    
    return pos, action, reward, successor_state

# Monte Carlo Control

In [129]:
def e_greedy(eps, pos, Q): 
    
    
    
    #episilon = the probability of randomly choosing an action
    # probablility of choosing the optimal action is 1 - eps.
    
   
    probability = round(random.uniform(0,1), 2)
    
    if probability <= eps: 
        action = random.randint(0,3)
    else: 
       
        rewards = list(Q[pos])
        action = rewards.index(max(rewards))
        
      
    return action

In [130]:
def first_visit_mc_control(num_iterations, discount): 
    
    Q = defaultdict(lambda: np.zeros(4))
    episode = []
    sa_visited = []
    sa_occurences = defaultdict(int)
    
    
    for it in range(1, num_iterations + 1): 
        
        is_terminal = False
        initial_state = (random.randint(0,3), random.randint(0,3))
        
        while not is_terminal:
            
            # the action must be chosen with the epsilon greedy policy. 
            initial_state, action, reward, successor_state = step_control(initial_state, Q)
            sa = (initial_state, action)
            episode.append((sa, reward))
            sa_occurences[sa] += 1
            initial_state = successor_state
            if initial_state in terminal_state: 
                is_terminal = True
            
        current_return = 0
        power = len(episode)
        for sa, reward in reversed(episode): 
                 
            if sa not in sa_visited: 
                    
                current_return += (discount ** power) * reward
                
                Q[sa[0]][sa[1]] += round((1 / sa_occurences[sa]) * (reward - Q[sa[0]][sa[1]]), 2)
                    
                sa_visited.append(sa)
                
                power -= 1    
    return Q
   
                
                
        
        

In [111]:
Q = first_visit_mc_control(2000, 0.7)

In [112]:
# construct policy map

grid = np.zeros((4,4))

for state in Q.keys(): 
    
    grid[state[0], state[1]] = max(Q[state])

grid

array([[ 0.  , -0.25, -0.5 , -0.06],
       [-0.2 , -0.25, -0.25, -0.11],
       [-1.  , -1.  , -1.  , -0.25],
       [-0.08, -0.2 , -0.5 ,  0.  ]])

# Temporal Difference Learning

In [131]:
def td_learning(num_iterations, alpha, discount): 
    
    Q = defaultdict(lambda: np.zeros(4))

    for it in range(1, num_iterations + 1): 
        
        is_terminal = False
        initial_state = (random.randint(0,3), random.randint(0,3))
    
        while not is_terminal:

            # the action must be chosen with the epsilon greedy policy. 
            initial_state, action, reward, successor_state = step(initial_state, Q)
    
            Q[initial_state][action] += alpha * (reward + discount * max(Q[successor_state] - Q[initial_state][action]))
        
            if initial_state in terminal_state: 
                is_terminal = True
            
            initial_state = successor_state
                                             
    return Q
                                        
                

In [132]:
Q = td_learning(2000, 0.5, 0.5)

In [133]:
grid = np.zeros((4,4))

for state in Q.keys(): 
    
    grid[state[0], state[1]] = max(Q[state])

grid

array([[ 0., -2., -4., -6.],
       [-2., -4., -6., -4.],
       [-4., -6., -4., -2.],
       [-6., -4., -2.,  0.]])

In [134]:
Q = td_learning(1000, 0.5, 0.5)


grid = np.zeros((4,4))

for state in Q.keys(): 
    
    grid[state[0], state[1]] = max(Q[state])

grid


array([[ 0.        , -2.        , -4.        , -5.99999984],
       [-2.        , -4.        , -5.99998012, -4.        ],
       [-4.        , -5.9999767 , -4.        , -2.        ],
       [-5.99999997, -4.        , -2.        ,  0.        ]])

In [135]:
Q = td_learning(500, 0.5, 0.5)

grid = np.zeros((4,4))

for state in Q.keys(): 
    
    grid[state[0], state[1]] = max(Q[state])

grid


array([[ 0.        , -2.        , -4.        , -5.99949074],
       [-2.        , -3.99999637, -5.98801623, -3.99999997],
       [-4.        , -5.99817449, -3.99999629, -2.        ],
       [-5.99989131, -4.        , -2.        ,  0.        ]])

In [136]:
Q = td_learning(650, 0.5, 0.5)

grid = np.zeros((4,4))

for state in Q.keys(): 
    
    grid[state[0], state[1]] = max(Q[state])

grid

array([[ 0.        , -2.        , -4.        , -5.99999076],
       [-2.        , -3.99999989, -5.99952092, -4.        ],
       [-4.        , -5.99918569, -3.99999994, -2.        ],
       [-5.99998697, -4.        , -2.        ,  0.        ]])

# Maze Environment

In [152]:
import numpy as np
import random

def create_maze_path(shape): 
    
    path_coordinates = []
    possible_coords = []
    coord_count = 0
    #generate a length for an array: 
    maze_length = random.randint(9,11);
   
    
    # generate a starting coordinate (always in the first row)
    starting_col = random.randint(0, shape[1] - 1); # has to be shape[1] because the length of a row is equal to the number of columns there are.
    path_coordinates.append((0, starting_col)) # append to maze path list
    coord_count += 1
    
    while coord_count < maze_length: 
    
        possible_coords = []
        
        current_pos = path_coordinates[-1] # holds the most recent path position
        if (current_pos[1] + 1 < shape[1]):  # if the previous column + 1 coordinate is less than the column shape, that means it can move to the right  
                #print(1)
                possible_coords.append((current_pos[0], current_pos[1] + 1))
                
            
        if (current_pos[1] - 1 >=  0): # if the previous column coordinate - 1 is less greater than or equal to 0, that means it can move to the left  
                
          
            possible_coords.append((current_pos[0], current_pos[1] - 1))
            
            
        if (current_pos[0] + 1 < shape[0]): # if the previous row coordinate + 1 is less than the row shape, that means it can move down
               
         
            possible_coords.append((current_pos[0] + 1, current_pos[1]))
            
            
        if (current_pos[0] - 1 >= 0): # if the previous row coordinate - 1 is greater than or equal to 0, that means it can move up. 
        
            possible_coords.append((current_pos[0] - 1,current_pos[1]))
        
        
        # get rid of any repeats in path_coordinates
        possible_coords = list(set(possible_coords))
        
        random_next_coordinate = random.randint(0, len(possible_coords) - 1)
        
        # so now that we don't want repeats, we need a way that we can generate another random indice. 
        if possible_coords[random_next_coordinate] not in path_coordinates: 

            path_coordinates.append(possible_coords[random_next_coordinate]) # add the new coordinate to the path.
        else: 
            coord_count -= 1
           
        coord_count += 1

    
    
    return path_coordinates, shape
    
def generate_maze(coords, shape): 
    
    #print(coords)
    maze = np.random.randint(1,2, size=(shape))
    all_coords = coords # extra_pos needs to come first so the ending position (goal remains clear)
        
    for coord in all_coords: 
        maze[coord[0], coord[1]] = 0
        
    goal = all_coords[-1]
            
    return maze, goal
    

def add_pos(coords, shape): 
    
    # you can only add an extra zero at some place IF it is available. 
    # how many dead ends can you add? that number can be generated, based on the size of the maze. note that NOT all have to be used up. I will only set a max number. 
    # first, generate an amount of extra positions you can have
    
    num_extra = random.randint(2,4)
    
    
    
    candidates = [] # holds all possible extra positions coordinates (with repeats)

    
    coord_count = 1 # because if it's 0, we can't do row - 1
    
    while coord_count <= (len(coords) - 2): 
        
        
        rand_pos = random.randint(0,len(coords) - 1)
        all_taken = True
        
        row = coords[rand_pos][0]
        
        col = coords[rand_pos][1]
       
        
        # need to check the bounds of the 2D maze
        if ( (row, col + 1) not in coords and (col + 1) < shape[1] ):
            candidates.append((row, col +1))
            
        if ( (row, col -1) not in coords and (col - 1) > 0):
            candidates.append((row, col-1))
            
        if ( (row + 1, col) not in coords and (row + 1) < shape[0]):
            candidates.append((row+1, col))
            
        if ( (row-1, col) not in coords and (row -1) > 0):
            candidates.append((row-1, col))
            
        coord_count += 1
    
    final_candidates = list(set(candidates)) # set of coordinates without repeats. 
    
    #print(final_candidates)
    #print(len(final_candidates))    
    final_num_extra = random.randint(1, len(final_candidates)) # get the number of extra dead ends you can have. 
    #print(final_num_extra)
    extra_pos = random.sample(final_candidates, final_num_extra)
    
    return extra_pos
    


In [154]:
coords, shape = create_maze_path((5,5))
#extra_pos = add_pos(coords, shape)
print(generate_maze(coords, shape))

(array([[1, 0, 0, 1, 1],
       [1, 0, 0, 1, 1],
       [1, 1, 0, 0, 1],
       [1, 1, 1, 0, 1],
       [0, 0, 0, 0, 1]]), (4, 0))
